In [255]:
import pandas as pd

In [256]:
# Function to calculate fiscal year in "YYYY-YYYY" format
def get_fiscal_year_range(date):
    start_month = 4
    if date.month >= start_month:
        return f"{date.year}-{date.year + 1}"
    else:
        return f"{date.year - 1}-{date.year}"

In [257]:
df_tax = pd.read_csv("../DATA/BRONZE/TaxReport/TaxReport_data.csv")

# Apply the function to add a new column for fiscal year
df_tax["fy"] = pd.to_datetime(df_tax["date"]).apply(get_fiscal_year_range)

df_tax = (
    df_tax.groupby(
        [
            "stock_name",
        ]
    )
    .agg({"pnl_amount": "sum"})
    .sort_values(by=["stock_name"])
    .reset_index()
)
print(df_tax["pnl_amount"].sum())
df_tax

10173.7


,stock_name,pnl_amount
0,BHAGERIA,29.50
1,BPCL,-5510.45
2,GOLDBEES,0.42
3,HERANBA,3944.00
4,IDEA,114.00
5,INFY,182.45
6,LICI,-3294.75
7,NIFTY,5373.75
8,NIFTYBEES,6.43
9,PNB,255.00


In [258]:
# Read the CSV file into a DataFrame
df_raw = pd.read_csv("../DATA/SILVER/TradeHistory/TradeHistory_data.csv")
# Use a lambda function with apply to update stock_name
df_raw["stock_name"] = df_raw["stock_name"].apply(
    lambda name: "NIFTY" if "NIFTY-" in name else name
)

# Calculate the total buy and sell quantities and prices
df_raw = (
    df_raw.groupby(["stock_name", "side"])
    .agg({"quantity": "sum", "amount": "sum", "price": "sum"})
    .reset_index()
)

# Pivot the data for a better structure
df_raw = df_raw.pivot(
    index=["stock_name"],
    columns="side",
    values=["quantity", "amount", "price"],
).fillna(0)

df_raw.columns = [
    "_".join(list(reversed(col))).strip().lower()
    for col in df_raw.columns.values
]
df_raw["buy_price"] = df_raw["buy_amount"] / df_raw["buy_quantity"]
df_raw["sell_price"] = df_raw["sell_amount"] / df_raw["sell_quantity"]
df_raw["pnl_amount"] = (df_raw["sell_price"] - df_raw["buy_price"]) * df_raw[
    "sell_quantity"
]
df_raw["holding_quantity"] = df_raw["buy_quantity"] - df_raw["sell_quantity"]
df_raw["investment_amount"] = df_raw["buy_price"] * df_raw["holding_quantity"]
df_raw["avg_price"] = df_raw["investment_amount"] / df_raw["holding_quantity"]
# Order by holding_quantity and stock_name
df_raw = df_raw.sort_values(by=["stock_name"]).fillna(0).reset_index()

round_cols = [
    "pnl_amount",
    "holding_quantity",
    "investment_amount",
    "avg_price",
]

df_raw[round_cols] = df_raw[round_cols].astype(float).round(2)

df_raw = df_raw[
    [
        "stock_name",
        "pnl_amount",
        "holding_quantity",
        "investment_amount",
        "avg_price",
    ]
]
df_raw[df_raw["holding_quantity"]!=0]

SyntaxError: invalid syntax (3855627239.py, line 55)

In [ ]:
df_raw = (
    df_raw.groupby(
        [
            "stock_name",
        ]
    )
    .agg({"pnl_amount": "sum"})
    .sort_values(by=["stock_name"])
    .reset_index()
)
print(df_raw["pnl_amount"].sum())
df_raw

26169.25


,stock_name,pnl_amount
0,BHAGERIA,29.50
1,BPCL,-5510.45
2,GOLDBEES,0.42
3,HERANBA,3944.00
4,IDEA,114.00
5,INFY,182.45
6,IRCTC,0.00
7,KPITTECH,0.00
8,LICI,-3294.75
9,MIRAE-ASSET-TAX-SAVER-DIRECT-GROWTH,0.00


In [ ]:
# Merge DataFrames on 'stock_name'
df_merged = pd.merge(df_raw, df_tax, on='stock_name', suffixes=('_raw', '_tax'), how='outer')

# Compare pnl_amount from both DataFrames
df_merged['pnl_match'] = df_merged['pnl_amount_raw'] == df_merged['pnl_amount_tax']

# Remove duplicate stock names, if necessary
# df_merged = df_merged.drop_duplicates(subset=['stock_name'])
df_merged.sort_values(by=["pnl_match","stock_name"]).fillna(0).reset_index(drop=True)

,stock_name,pnl_amount_raw,pnl_amount_tax,pnl_match
0,IRCTC,0.00,0.00,False
1,KPITTECH,0.00,0.00,False
2,MIRAE-ASSET-TAX-SAVER-DIRECT-GROWTH,0.00,0.00,False
3,NIFTY,22338.86,5373.75,False
4,SBI-MAGNUM-TAXGAIN-SCHEME-DIR-GR,0.00,0.00,False
5,TATAMOTORS,0.00,0.00,False
6,TATAPOWER,1693.29,2662.85,False
7,BHAGERIA,29.50,29.50,True
8,BPCL,-5510.45,-5510.45,True
9,GOLDBEES,0.42,0.42,True


In [ ]:
# filepath of the CSV file
filepath = "../DATA/GOLD/ProfitLoss/ProfitLoss_data.csv"

# Read the CSV file into a DataFrame
df_pnl = pd.read_csv(filepath)
df_pnl = df_pnl[df_pnl["stock_name"].isin(["NIFTY-PE-24650-18JUL2024"])]
df_pnl

,datetime,exchange,segment,stock_name,side,quantity,open_price,open_amount,close_price,close_amount,pnl_amount,pnl_percentage
81,2024-07-15 14:18:31,FON,FO,NIFTY-PE-24650-18JUL2024,LONG,50.0,115.25,5762.5,115.75,5787.5,25.00,0.43
82,2024-07-18 09:47:40,FON,FO,NIFTY-PE-24650-18JUL2024,LONG,50.0,77.68,3884.0,70.25,3512.5,-371.25,-9.56
83,2024-07-18 15:30:00,FON,FO,NIFTY-PE-24650-18JUL2024,EXPIRED,100.0,3.00,300.0,0.00,0.0,-300.00,-100.00


In [ ]:
# Calculate the total buy and sell quantities and prices
df_pnl = df_pnl.groupby(["segment", "stock_name"]).agg({"pnl_amount": "sum"})

df_pnl = df_pnl.sort_values(by=["segment", "stock_name"])
# Reset the index to get stock names as a column
df_pnl.reset_index(inplace=True)
df_pnl

,segment,stock_name,pnl_amount
0,FO,NIFTY-PE-24650-18JUL2024,-646.25


In [ ]:
# Rename the pnl_amount columns to distinguish them
df1 = df[["stock_name", "pnl_amount"]].rename(
    columns={"pnl_amount": "code_pnl"}
)
df2 = df_pnl[["stock_name", "pnl_amount"]].rename(
    columns={"pnl_amount": "my_pnl"}
)

# Perform an outer merge on the stock_name column
merged_df = pd.merge(df1, df2, on="stock_name", how="outer").fillna(0)
merged_df = merged_df[merged_df["code_pnl"] != merged_df["my_pnl"]]
merged_df

,stock_name,code_pnl,my_pnl
0,BHAGERIA,29.500000,0.00
1,BPCL,-5510.450000,0.00
2,GOLDBEES,0.420000,0.00
3,HERANBA,3944.000000,0.00
4,IDEA,114.000000,0.00
5,INFY,182.450000,0.00
8,LICI,-3294.750000,0.00
10,NIFTY,22347.142027,0.00
11,NIFTY-PE-24650-18JUL2024,0.000000,-646.25
12,PNB,255.000000,0.00
